# Linear and Integer Programming Breakout Notebook



If you don't have cvxpy, you should run the following block. It may take a few minutes.  Make sure you did `conda activate vrdi` or the equivalent before starting up Jupyter.



In [ ]:
# conda install cvxpy
import sys
!conda config --add channels oxfordcontrol
!conda install -c conda-forge lapack --yes
!conda install -c cvxgrp cvxpy=1.0.24 --yes
!conda install --yes --prefix {sys.prefix} cvxpy

In [ ]:
# Imports
import numpy as np
import cvxpy as cp
import networkx as nx
import itertools
import time
from IPython.display import clear_output
from itertools import chain, combinations



from cvxpy.reductions.solvers.defines import INSTALLED_SOLVERS
print(INSTALLED_SOLVERS)

## The Diet Problem

Let's start by creating our variables and writing down the objective function:

In [ ]:
## The diet problem
x = cp.Variable(6)#, integer = True)

objective = cp.Minimize(
     3*x[0] + 
    24*x[1] + 
    13*x[2] + 
     9*x[3] + 
    20*x[4] +
    19*x[5]
)

...and add the constraints.

In [ ]:
constraints = [
    (110*x[0] + 205*x[1] + 160*x[2] + 160*x[3] + 420*x[4] + 260*x[5] >= 2000),
    (4*x[0] + 32*x[1] + 13*x[2] + 8*x[3] + 4*x[4]  + 14*x[5] >= 55),
    (2*x[0] + 12*x[1] + 54*x[2] + 285*x[3] + 22*x[4]  + 80*x[5] >= 800),
    (0 <= x[0]), (x[0] <= 4),
    (0 <= x[1]), (x[1] <= 3),
    (0 <= x[2]), (x[2] <= 2),
    (0 <= x[3]), (x[3] <= 8),
    (0 <= x[4]), (x[4] <= 2),
    (0 <= x[5]), (x[5] <= 2)
]

The problem is to optimize `objective` subject to `constraints`, and `cvxpy` can solve this problem!

In [ ]:
prob = cp.Problem(objective,constraints)
prob.solve()

# Print result.
print("\nThe optimal value is {}".format(np.round_(prob.value,3)))
print("A solution x is \n{}".format(np.round_(x.value,3)))

## Shortest s-t Path

It's going to be convenient to represent the graph as a collection of triples `(u,v,w)` where `u,v` is an arc in the graph and `w` is the weight of the arc.

In [ ]:







edges = [
    (1,2,8),  #edges[0]
    (1,3,15), #edges[1]
    (1,4,9),  #edges[2]
    (2,3,13), #edges[3]
    (3,5,6),  #edges[4]
    (3,7,5),  #edges[5]
    (4,5,13), #edges[6]
    (5,6,7),  #edges[7]
    (6,8,4),  #edges[8]
    (7,6,9),  #edges[9]
    (7,8,18)  #edges[10]
]

# we need a binary variable for each edge
#x_e = cp.Variable(len(edges), integer=True)
x_e = cp.Variable(len(edges), integer = True)

The objective is to minimize the total weights of the edges carrying flow...

In [ ]:
wts = np.array([e[2] for e in edges])

objective = cp.Minimize( wts.T @ x_e)

...subject to the constraints that one unit must leave s, one unit must arrive at t, and the net flow at all other vertices is zero.

In [ ]:
# for each node u, the total value of the flow leaving 
# it minus the total value of the flow into it should be
# 1 if u is the source, -1 if u is the destination, and 0 otherwise


# constraints
constraints = []
# variables positive
constraints += [(x_e >= 0)]

# one unit leaving the source (u=1)
constraints += [(x_e[0] + x_e[1] + x_e[2] == 1)]

# one unit entering the sink (u=8)
constraints += [(-x_e[8] - x_e[10] == -1)]

# and an inscrutible list comprehension
constraints += [(sum([x_e[j] for j in range(11) if edges[j][1] == v]) + 
                 cp.sum([-x_e[j] for j in range(11) if edges[j][0] == v]) == 0 ) 
                for v in range(2,8)]


## this is the same, more readable, more lines

# # one unit leaving the source (u=1)
# constraints += [(x_e[0] + x_e[1] + x_e[2] == 1)]

# # one unit entering the sink (u=8)
# constraints += [(-x_e[8] - x_e[10] == -1)]

# # u=2
# constraints += [(x_e[0] - x_e[3]) == 0]

# # u=3
# constraints += [(x_e[1] + x_e[3] - x_e[4]) - x_e[5] == 0]

# # u=4
# constraints += [(x_e[2] - x_e[6]) == 0]

# # u=5
# constraints += [(x_e[4] + x_e[6]) - x_e[7] == 0]

# # u=5
# constraints += [(x_e[4] + x_e[6]) - x_e[7] == 0]

# # u=6
# constraints += [(x_e[7] + x_e[9]) - x_e[8] == 0]

# # u=7
# constraints += [(x_e[5] - x_e[9]) - x_e[10] == 0]

Let's solve!  After you solve the Integer program, go back and remove the stipulation that the `Variable`s are integers and check that you get the same objective value.

In [ ]:
print("solving")

prob = cp.Problem(objective,constraints)
prob.solve()

# Print result.
print("\nThe optimal value is {}".format(np.round_(prob.value,3)))
print("A solution x is \n {}".format(np.round_(x_e.value,3)))

This is the second attempt, and is actually the dual of the previous one.

The objective is to maximize the total distance that we can pull t from s...

In [ ]:
edges = [
    (1,2,8),  #edges[0]
    (1,3,15), #edges[1]
    (1,4,9),  #edges[2]
    (2,3,13), #edges[3]
    (3,5,6),  #edges[4]
    (3,7,5),  #edges[5]
    (4,5,13), #edges[6]
    (5,6,7),  #edges[7]
    (6,8,4),  #edges[8]
    (7,6,9),  #edges[9]
    (7,8,18)  #edges[10]
]


# a binary variable for each node
x_v = cp.Variable(8, integer = True)

objective = cp.Maximize( x_v[7])

...subject to the constraints that the distances between pairs of vertices follow a triangle inequality with respect to the edge weights.

In [ ]:
# constraints
constraints = []
# s is distance 0 from itself
constraints += [(x_v[0]) == 0]

# the distance from s to any vertex v is at most the distance to a 
# predecessor u of v plus the length of the arc connecting u to v
for e in edges:
    constraints += [(x_v[e[1]-1] - x_v[e[0]-1]  <= e[2]  )]

And solve! Again, check that relaxing this to a linear, rather than integer, program gives the same objective value.

In [ ]:
print("solving")
tic = time.time()
prob = cp.Problem(objective,constraints)
prob.solve()

# Print result.
print("\nThe optimal value is {}".format(np.round_(prob.value,3)))
print("A solution x is \n {}".format(np.round_(x_e.value,3)))

## Here's some demo code for the MA problem

In [ ]:
import geopandas as gpd
from gerrychain.graph import Graph

# setup -- SLOW

df = gpd.read_file("https://github.com/mggg-states/MA-shapefiles/raw/master/MA_no_islands_12_16.zip")
   
graph = Graph.from_geodataframe(df, reproject=True,ignore_errors=True)
graph.add_data(df,list(df))

In [ ]:
pops = [int(p) for p in list(nx.get_node_attributes(graph,"Population").values())]
r_margin = [int(r.replace(",",""))-int(d.replace(",","")) for r,d in zip(list(nx.get_node_attributes(graph,"SEN12R").values()), 
                               list(nx.get_node_attributes(graph,"SEN12D").values()))]

In [ ]:
print(list(df))
print(len(pops))

In [ ]:
# Boolean variables assigning units to districts
d1 = cp.Variable(len(pops), boolean=True)
d2 = cp.Variable(len(pops),boolean=True)
d3 = cp.Variable(len(pops),boolean=True)
d4 = cp.Variable(len(pops),boolean=True)
d5 = cp.Variable(len(pops),boolean=True)
d6 = cp.Variable(len(pops),boolean=True)
d7 = cp.Variable(len(pops),boolean=True)
d8 = cp.Variable(len(pops),boolean=True)
d9 = cp.Variable(len(pops),boolean=True)


# u makes sure that each unit is assigned to at most one district
u = cp.Variable(len(pops),boolean=True)


## a lot of these are "extra" and are there to show you how you might extend this
## to solve other problems


# cast the lists to arrays
pops = np.array(pops)
r_margin = np.array(r_margin)


constraints = []

# the population of our district shouldn't be too big or too small
constraints += [(pops.T @ d1 >= 726500)]
constraints += [(pops.T @ d1 <= 728500)]

constraints += [(pops.T @ d2 >= 726500)]
constraints += [(pops.T @ d2 <= 728500)]

constraints += [(pops.T @ d3 >= 726500)]
constraints += [(pops.T @ d3 <= 728500)]

constraints += [(pops.T @ d4 >= 726500)]
constraints += [(pops.T @ d4 <= 728500)]
constraints += [(pops.T @ d5 >= 726500)]
constraints += [(pops.T @ d5 <= 728500)]
constraints += [(pops.T @ d6 >= 726500)]
constraints += [(pops.T @ d6 <= 728500)]
constraints += [(pops.T @ d7 >= 726500)]
constraints += [(pops.T @ d7 <= 728500)]
constraints += [(pops.T @ d8 >= 726500)]
constraints += [(pops.T @ d8 <= 728500)]
constraints += [(pops.T @ d9 >= 726500)]
constraints += [(pops.T @ d9 <= 728500)]

constraints += [d1+d2+d3+d4+d5+d6+d7+d8+d9 <= u]



# double check that none of the constraints are trivial    
constraints = [ c for c in constraints if c!=True and c!=False]

# we want to maximize the total republican margin in the district
objective = cp.Maximize( r_margin.T @ d1)# + r_margin.T @ d2 + r_margin.T @ d3 + r_margin.T @ d4 + r_margin.T @ d5 + r_margin.T @ d6) 





print("solving")
prob = cp.Problem(objective,constraints)
prob.solve()

# Print result.
print("\nThe optimal value is".format(int( prob.value)))
print("A solution x is probably too big to print, but you can do it in the next block...")


In [ ]:
print(list(d1.value))

In [ ]:
print(r_margin.T @ d1.value)
print(r_margin.T @ d2.value)
print(r_margin.T @ d3.value)
print(r_margin.T @ d4.value)
print(r_margin.T @ d5.value)
print(r_margin.T @ d6.value)
print(r_margin.T @ d7.value)
print(r_margin.T @ d8.value)
print(r_margin.T @ d9.value)